In [1]:
from omegaconf import OmegaConf
import os
import polars as pl
import pandas as pd

In [2]:
def load_data_paths(path_name):
    notebooks_dir = os.getcwd()
    base_dir = os.path.abspath(os.path.join(notebooks_dir, ".."))
    config_path = os.path.join(base_dir, "config.yaml")
    config = OmegaConf.load(config_path)
    paths = config[path_name]
    dos_df_path = os.path.join(base_dir,paths.dos_df).replace("\\", "/")
    fuzzy_df_path = os.path.join(base_dir, paths.fuzzy_df).replace("\\", "/")
    attack_free_df_path = os.path.join(base_dir, paths.attack_free_df).replace("\\", "/")
    return dos_df_path,fuzzy_df_path,attack_free_df_path

In [3]:
def load_datasets_using_pl():
    dos_df_path,fuzzy_df_path,attack_free_df_path=load_data_paths("out_paths")
    dos_df = pl.read_csv(dos_df_path)
    fuzzy_df = pl.read_csv(fuzzy_df_path)
    attack_free_df = pl.read_csv(attack_free_df_path)
    return dos_df, fuzzy_df, attack_free_df

In [4]:
def load_datasets_using_pd():
    dos_df_path,fuzzy_df_path,attack_free_df_path=load_data_paths("out_paths")
    dos_df = pd.read_csv(dos_df_path)
    fuzzy_df = pd.read_csv(fuzzy_df_path)
    attack_free_df = pd.read_csv(attack_free_df_path)
    return dos_df, fuzzy_df, attack_free_df